<a href="https://colab.research.google.com/github/testgithubprecious/Ml_projects/blob/main/NeuralArch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install if not already: pip install torch pandas numpy scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

# ------------------------------
# Sample implicit feedback data
# ------------------------------
data = {
    "user_id": [0, 0, 1, 1, 2, 2, 3, 4],
    "item_id": [1, 2, 2, 3, 1, 3, 0, 1],
    "rating":  [1, 1, 1, 1, 1, 1, 1, 1]  # Implicit feedback (1 = interaction)
}
df = pd.DataFrame(data)

# ------------------------------
# Unique user/item counts
# ------------------------------
num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()

# Train/test split
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)

# ------------------------------
# Neural Collaborative Filtering Model
# ------------------------------
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=8):
        super().__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, user_ids, item_ids):
        u_embed = self.user_embedding(user_ids)
        i_embed = self.item_embedding(item_ids)
        x = torch.cat([u_embed, i_embed], dim=1)
        return self.mlp(x)

# ------------------------------
# Instantiate model, loss, optimizer
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NCF(num_users, num_items).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ------------------------------
# Convert training data to tensors
# ------------------------------
user_tensor = torch.tensor(train_data["user_id"].values, dtype=torch.long, device=device)
item_tensor = torch.tensor(train_data["item_id"].values, dtype=torch.long, device=device)
label_tensor = torch.tensor(train_data["rating"].values, dtype=torch.float32, device=device)

# ------------------------------
# Train (1 epoch demo)
# ------------------------------
model.train()
for epoch in range(1):
    preds = model(user_tensor, item_tensor).squeeze()
    loss = criterion(preds, label_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"✅ Epoch {epoch+1} - Loss: {loss.item():.4f}")

# ------------------------------
# Test prediction
# ------------------------------
model.eval()
with torch.no_grad():
    test_user = torch.tensor(test_data["user_id"].values, dtype=torch.long, device=device)
    test_item = torch.tensor(test_data["item_id"].values, dtype=torch.long, device=device)
    test_preds = model(test_user, test_item).squeeze()
    print("\n📢 Test Predictions (interaction scores):", test_preds.tolist())